In [13]:
import json
import pandas as pd
from typing import List, Dict, Any


class Normalizer:
    def __init__(self, relations, fds, mvds, target_nf):
        self.relations = relations
        self.fds = fds
        self.mvds = mvds
        self.target_nf = target_nf
        self.normalized_relations = []
        self.table_counter = 1
        self.join_dependencies = []

    def identify_dependencies(self):
      
        for fd in self.fds:
            lhs = fd['lhs']
            rhs = fd['rhs']
            if len(lhs) > 1 and len(rhs) > 1:
                self.join_dependencies.append({
                    'lhs': lhs,
                    'rhs': rhs
                })

    def normalize(self):

        # dependencies before starting normalization
        self.identify_dependencies()


        for relation in self.relations:
            print(f"\nNormalizing table: {relation['table_name']}")
            
            # Track decompositions
            nf_history = {'1NF': [], '2NF': [], '3NF': [], 'BCNF': [], '4NF': [], '5NF': []}

            # Display the original relation
            orginal_table = relation.copy()
            print("Original table before normalization:")
            self.print_table(relation, "Original")

            # Always apply 1NF
            self.apply_1NF(relation)
            #self.decompose_1NF(relation)
            #self.display_and_store_decompositions(relation, nf_history, "1NF")
            if self.apply_1NF:
                self.display_and_store_decompositions(relation, nf_history, "1NF")
                self.print_table(relation, "Original")

            if self.target_nf == "1NF":
                continue  # Stop after applying 1NF

            # Apply 2NF and display tables if needed
            if self.target_nf in ["2NF", "3NF", "BCNF", "4NF", "5NF"]:
                #self.apply_2NF(relation)
                self.apply_2NF(relation, nf_history)
                #print("Your intial table's highest normal form was 1NF.")
                if not self.apply_2NF:
                    self.display_and_store_decompositions(relation, nf_history, "2NF")
                    self.print_table(relation, "Original")

            if self.target_nf == "2NF":
                continue  # Stop after applying 2NF

            # Apply 3NF and display tables if needed
            if self.target_nf in ["3NF", "BCNF", "4NF", "5NF"]:
                self.apply_3NF(relation, nf_history)
                if not self.apply_3NF:
                    self.display_and_store_decompositions(relation, nf_history, "3NF")
               

            if self.target_nf == "3NF":
                continue  # Stop after applying 3NF

            # Apply BCNF and display tables if needed
            if self.target_nf in ["BCNF", "4NF", "5NF"]:
                self.apply_bcnf(relation, nf_history)
                if not self.apply_bcnf:
                    self.display_and_store_decompositions(relation, nf_history, "BCNF")
              

            if self.target_nf == "BCNF":
                continue  # Stop after applying BCNF

            if self.target_nf in ["4NF", "5NF"]:
                self.apply_4NF(relation, nf_history)
                if not self.apply_4NF:
                    self.display_and_store_decompositions(relation, nf_history, "4NF")
                         

            # Apply 4NF if target is 4NF
            if self.target_nf == "4NF":
                continue

            if self.target_nf == "5NF":
                self.apply_5NF(relation, nf_history)
                if not self.apply_5NF:
                    self.display_and_store_decompositions(relation, nf_history, "5NF")
                else:
                    print("All tables are already in 5NF.")
            if self.target_nf == "5NF":
                continue

        
        print("\nAll decompositions for each normal form level:")
        for nf_level, tables in nf_history.items():
            print(f"\nDecompositions after applying {nf_level}:")
            for table in tables:
                # Print the table name with normal form levels applied
                table_name = f"{table['table_name']}_{nf_level}"
                
                # Output the table's name, attributes, and primary key
                print(f"Decomposed table {table_name}:")
                print(f"Attributes: {table['attributes']}")
                print(f"Primary Key: {table['primary_key']}\n")

        return self.normalized_relations

    def display_and_store_decompositions(self, relation, nf_history, nf_level):
       
        decompositions = []
        for table in self.normalized_relations:
            if table['table_name'].startswith(relation['table_name']):
                # Store a copy of the table to preserve current state
                decompositions.append(table.copy())
                print(f"\nNew table {table['table_name']} after applying {nf_level}:")
                df = pd.DataFrame(table['rows'], columns=table['attributes'])
                print(df.to_string(index=False))
        
        # Append decompositions to history without clearing normalized relations
        nf_history[nf_level].extend(decompositions)

    def split_rows(self, rows, non_atomic_attrs, attributes):
        split_rows = []

        # Convert rows from list of lists to list of dicts for easier processing
        dict_rows = [dict(zip(attributes, row)) for row in rows]

        for row in dict_rows:
            # Get base row containing only primary keys and non-non_atomic attributes
            base_row = {attr: row.get(attr, None) for attr in attributes if attr not in non_atomic_attrs}
            
            # Process each non-atomic attribute
            for attr in non_atomic_attrs:
                if attr in row and '{' in row[attr]:  # Confirm it's non-atomic
                    # Split the value of the non-atomic attribute
                    values = self.Get_values(row[attr])
                    for value in values:
                        new_row = base_row.copy()
                        new_row[attr] = value.strip()  # Ensure individual atomic value
                        split_rows.append(new_row)
                else:
                    # If attribute is already atomic, simply add it to the base row
                    new_row = base_row.copy()
                    new_row[attr] = row.get(attr, None)
                    split_rows.append(new_row)

        return split_rows

    def Get_values(self, non_atomic_value):
        # Ensure the value is always treated as a set of individual elements
        if isinstance(non_atomic_value, str) and non_atomic_value.startswith('{') and non_atomic_value.endswith('}'):
            inner_content = non_atomic_value[1:-1]  # Remove braces
            return [value.strip() for value in inner_content.split(',')]
        elif isinstance(non_atomic_value, str):  # Handle values without braces
            return [non_atomic_value.strip()]
        else:
            return [str(non_atomic_value).strip()]  # Fallback for any unexpected format


    def Get_rows(self, rows, lhs, rhs):
      
        extracted_rows = []
        for row in rows:
            # Create a subset of row data for the new table with only lhs + rhs attributes
            new_row = {attr: row[attr] for attr in lhs + rhs if attr in row}
            if new_row:  # Ensure new_row is non-empty
                extracted_rows.append(new_row)
        return extracted_rows

    def apply_1NF(self, relation):
       
        non_atomic = relation.get("non_atomic", [])
        decomposed_tables = []
        
        if non_atomic:
            print(f"\nTable {relation['table_name']} violates 1NF (non-atomic attributes: {non_atomic}). Decomposing.")
            
            for attr in non_atomic:
                new_attributes = relation['primary_key'] + [attr]
                split_rows = self.split_rows(relation['rows'], [attr], relation['attributes'])  # Pass the attributes
                new_relation = {
                    'table_name': f"{relation['table_name']}{self.table_counter}",
                    'attributes': new_attributes,
                    'rows': split_rows,
                    'primary_key': relation['primary_key']
                }
                decomposed_tables.append(new_relation)
                self.normalized_relations.append(new_relation)
                self.table_counter += 1
                
                # DEBUG
                print("Decomposing non-atomic attribute:", attr)
                print("Generated table:", new_relation['table_name'])
            
            # Identify atomic attribute indices and reformat rows
            atomic_attributes = [attr for attr in relation['attributes'] if attr not in non_atomic]
            atomic_indices = [relation['attributes'].index(attr) for attr in atomic_attributes]
            
            # Reformat rows to include only atomic attribute values in correct order
            relation['rows'] = [[row[idx] for idx in atomic_indices] for row in relation['rows']]
            
            # Update attributes to reflect only atomic attributes
            relation['attributes'] = atomic_attributes

            return decomposed_tables, True
        
        else:
            print(f"\nTable {relation['table_name']} does not violate 1NF.")
            return False

    def apply_2NF(self, relation, nf_history):
        original_table_name = relation['table_name']  # Store original table name for reference
        all_tables = [relation] + self.normalized_relations  # Include original and 1NF decompositions
        decomposed_tables = []
        removed_attributes = set()  # Track attributes removed from the original table

        for table in all_tables:
            # Check if the current table is the original table
            is_original_table = (table['table_name'] == original_table_name)
            
            partial_dependency_found = False  # Reset for each table
            relevant_fds = [
                fd for fd in self.fds
                if set(fd['lhs']).issubset(set(table['attributes'])) and set(fd['rhs']).issubset(set(table['attributes']))
            ]

            for fd in relevant_fds:
                lhs, rhs = fd['lhs'], fd['rhs']
                primary_key = table['primary_key']

                # Check for a partial dependency
                if set(lhs).issubset(set(primary_key)) and not set(rhs).issubset(set(primary_key)):
                    partial_dependency_found = True
                    print(f"\nTable {table['table_name']} violates 2NF (partial dependency: {fd}). Decomposing.")

                    # Create new relation based on partial dependency
                    new_relation = {
                        'table_name': f"{table['table_name']}_2NF_{self.table_counter}",
                        'attributes': list(set(lhs + rhs)),
                        'primary_key': lhs,
                        'rows': self.Get_rows(table['rows'], lhs, rhs),
                    }
                    decomposed_tables.append(new_relation)
                    self.normalized_relations.append(new_relation)
                    self.table_counter += 1

                    # Display the new decomposed table immediately after creation
                    print(f"Decomposed table {new_relation['table_name']}:")
                    print("Attributes:", new_relation['attributes'])
                    #print("Rows:", new_relation['rows'])
                    print("Primary Key:", new_relation['primary_key'])

                    nf_history['2NF'].append(new_relation)


                    # Remove RHS attributes of this dependency from the current table and track removed attributes
                    for attr in rhs:
                        if attr not in removed_attributes:
                            removed_attributes.add(attr)
                            table['attributes'].remove(attr)

            # Update rows of the original table only if a partial dependency was found
            if partial_dependency_found and is_original_table:
                print(f"\nUpdating original table {table['table_name']} after decomposition:")
                table['rows'] = [
                    [value for i, value in enumerate(row) if i < len(table['attributes']) and table['attributes'][i] not in removed_attributes]
                    for row in table['rows']
                ]

                # Output the updated original table after attribute removal
                print(f"Updated original table {table['table_name']}:")
                print("Attributes:", table['attributes'])
                #print("Rows:", table['rows'])
                print("Primary Key:", table['primary_key'])


        else:
            print("All tables are in 2NF.")

        return decomposed_tables if decomposed_tables else False

    def remove_partial_rows(self, rows, decomposed_tables, primary_key):
       
        # Use the original table's attributes to get indices
        original_attributes = decomposed_tables[0]['attributes']  # Replace with the original table's attributes list
        key_indices = [original_attributes.index(attr) for attr in primary_key if attr in original_attributes]

        filtered_rows = []
        seen_keys = set()

        for row in rows:
            # Generate the primary key tuple based on indices
            row_key = tuple(row[idx] for idx in key_indices if idx < len(row))
            
            # Only add rows with unique primary keys
            if row_key not in seen_keys:
                seen_keys.add(row_key)
                filtered_rows.append(row)

        return filtered_rows

    def apply_3NF(self, relation, nf_history):
        original_table_name = relation['table_name']  # Store original table name for reference
        all_tables = [relation] + self.normalized_relations  # Include original and 2NF decompositions
        decomposed_tables = []
        removed_attributes = set()  # Track attributes removed from the original table

        for table in all_tables:
            # Check if the current table is the original table
            is_original_table = (table['table_name'] == original_table_name)
            
            transitive_dependency_found = False  # Reset for each table
            relevant_fds = [
                fd for fd in self.fds
                if set(fd['lhs']).issubset(set(table['attributes'])) and set(fd['rhs']).issubset(set(table['attributes']))
            ]

            for fd in relevant_fds:
                lhs, rhs = fd['lhs'], fd['rhs']
                primary_key = table['primary_key']

                # Check for a transitive dependency
                if not set(lhs).issubset(set(primary_key)) and any(attr not in primary_key for attr in rhs):
                    transitive_dependency_found = True
                    print(f"\nTable {table['table_name']} violates 3NF (transitive dependency: {fd}). Decomposing.")

                    # Create new relation based on transitive dependency
                    new_relation = {
                        'table_name': f"{table['table_name']}_3NF_{self.table_counter}",
                        'attributes': list(set(lhs + rhs)),
                        'primary_key': lhs,
                        'rows': self.Get_rows(table['rows'], lhs, rhs),
                    }
                    decomposed_tables.append(new_relation)
                    self.normalized_relations.append(new_relation)
                    self.table_counter += 1

 
                    print(f"Decomposed table {new_relation['table_name']}:")
                    print("Attributes:", new_relation['attributes'])
                    #print("Rows:", new_relation['rows'])
                    print("Primary Key:", new_relation['primary_key'])

                    nf_history['3NF'].append(new_relation)

                    # Remove RHS attributes of this dependency from the current table and track removed attributes
                    for attr in rhs:
                        if attr not in removed_attributes:
                            removed_attributes.add(attr)
                            table['attributes'].remove(attr)

            # Update rows of the original table only if a transitive dependency was found
            if transitive_dependency_found and is_original_table:
                print(f"\nUpdating original table {table['table_name']} after decomposition:")
                table['rows'] = [
                    [value for i, value in enumerate(row) if i < len(table['attributes']) and table['attributes'][i] not in removed_attributes]
                    for row in table['rows']
                ]

                # Output the updated original table after attribute removal
                print(f"Updated original table {table['table_name']}:")
                print("Attributes:", table['attributes'])
                #print("Rows:", table['rows'])
                print("Primary Key:", table['primary_key'])

        else:
            print("All tables are in 3NF.")

        return decomposed_tables if decomposed_tables else False

    def apply_bcnf(self, relation, nf_history):
        original_table_name = relation['table_name']  # Store original table name for reference
        all_tables = [relation] + self.normalized_relations  # Include original and prior decompositions
        decomposed_tables = []
        removed_attributes = set()  # Track attributes removed from the original table

        for table in all_tables:
            is_original_table = (table['table_name'] == original_table_name)  # Check if this is the original table
            bcnf_violation_found = False  # Reset for each table
            relevant_fds = [
                fd for fd in self.fds
                if set(fd['lhs']).issubset(set(table['attributes'])) and set(fd['rhs']).issubset(set(table['attributes']))
            ]

            for fd in relevant_fds:
                lhs, rhs = fd['lhs'], fd['rhs']
                primary_key = table['primary_key']
                
                # Check for a BCNF violation (LHS is not a superkey)
                if not set(lhs).issuperset(set(primary_key)):
                    bcnf_violation_found = True
                    print(f"\nTable {table['table_name']} violates BCNF (dependency: {fd}). Decomposing.")

                    new_relation = {
                        'table_name': f"{table['table_name']}_BCNF_{self.table_counter}",
                        'attributes': list(set(lhs + rhs)),
                        'primary_key': lhs,
                        'rows': self.Get_rows(table['rows'], lhs, rhs),
                    }
                    decomposed_tables.append(new_relation)
                    self.normalized_relations.append(new_relation)
                    self.table_counter += 1

                    # Display the new decomposed table immediately after creation
                    print(f"Decomposed table {new_relation['table_name']}:")
                    print("Attributes:", new_relation['attributes'])
                    #print("Rows:", new_relation['rows'])
                    print("Primary Key:", new_relation['primary_key'])

                    nf_history['BCNF'].append(new_relation)

                    # Remove RHS attributes of this dependency from the current table and track removed attributes
                    for attr in rhs:
                        if attr not in removed_attributes:
                            removed_attributes.add(attr)
                            table['attributes'].remove(attr)

            # Update rows of the original table if a BCNF violation was found
            if bcnf_violation_found and is_original_table:
                print(f"\nUpdating original table {table['table_name']} after decomposition:")
                table['rows'] = [
                    [value for i, value in enumerate(row) if i < len(table['attributes']) and table['attributes'][i] not in removed_attributes]
                    for row in table['rows']
                ]

                # Output the updated original table after attribute removal
                print(f"Updated original table {table['table_name']}:")
                print("Attributes:", table['attributes'])
                print("Rows:", table['rows'])
                print("Primary Key:", table['primary_key'])

        # Return the decomposed tables or indicate all are in BCNF
        if decomposed_tables:
            print("\nFinal decomposed tables in BCNF:")
            for decomposed_table in decomposed_tables:
                print(f"Table {decomposed_table['table_name']}:")
                print("Attributes:", decomposed_table['attributes'])
                #print("Rows:", decomposed_table['rows'])
                print("Primary Key:", decomposed_table['primary_key'])
        else:
            print("All tables are in BCNF.")

        return decomposed_tables if decomposed_tables else False

    def apply_4NF(self, relation, nf_history):
        all_tables = [relation] + self.normalized_relations  # Include original and 1NF decompositions
        decomposed_tables = []

        for table in all_tables:
            multi_value_dependency_found = False  # Reset for each table
            candidate_keys = self.get_candidate_keys(table)  # Get candidate keys for the current table

            for mvd in self.mvds:
                lhs, rhs = mvd['lhs'], mvd['rhs']

                # Check for a violation of 4NF
                if not self.is_superkey(lhs, table['attributes'], candidate_keys):
                    multi_value_dependency_found = True
                    print(f"\nTable {table['table_name']} violates 4NF (MVD: {mvd}). Decomposing.")

                    # Create new relation based on MVD
                    new_relation = {
                        'table_name': f"{table['table_name']}_4NF_{self.table_counter}",
                        'attributes': list(set(lhs + rhs)),
                        'primary_key': lhs,
                        'rows': self.Get_rows(table['rows'], lhs, rhs),
                    }

                    # Display the newly created relation's structure
                    print(f"Decomposed table {new_relation['table_name']}:")
                    print("Attributes:", new_relation['attributes'])
                    #print("Rows:", new_relation['rows'])
                    print("Primary Key:", new_relation['primary_key'])

                    decomposed_tables.append(new_relation)
                    self.normalized_relations.append(new_relation)
                    self.table_counter += 1

                    nf_history['4NF'].append(new_relation)

                    # Remove RHS attributes of this MVD from the current table
                    table['attributes'] = [attr for attr in table['attributes'] if attr not in rhs]


        return decomposed_tables if decomposed_tables else False

    def get_candidate_keys(self, relation):
      
        candidate_keys = []
        
        # Start with primary key as a candidate key
        primary_key = relation['primary_key']
        candidate_keys.append(primary_key)

        # Function to check if a set of attributes can determine other attributes
        def can_determine(lhs, attributes):
            closure = set(lhs)
            while True:
                new_attributes = closure.copy()
                for fd in self.fds:
                    if set(fd['lhs']).issubset(closure):
                        new_attributes.update(fd['rhs'])
                if new_attributes == closure:
                    break
                closure = new_attributes
            return closure == set(attributes)

        # Find all combinations of attributes that can serve as candidate keys
        attributes = relation['attributes']
        from itertools import combinations

        # Iterate through all combinations of attributes
        for r in range(1, len(attributes) + 1):
            for combo in combinations(attributes, r):
                # Check if the combo can determine all attributes
                if can_determine(combo, attributes):
                    # Check if the combo is minimal
                    is_minimal = True
                    for key in candidate_keys:
                        if set(key).issubset(set(combo)):
                            is_minimal = False
                            break
                    if is_minimal:
                        candidate_keys.append(combo)

        # Remove duplicates and non-minimal candidate keys
        unique_candidate_keys = []
        for key in candidate_keys:
            if not any(set(key).issubset(set(other_key)) for other_key in unique_candidate_keys):
                unique_candidate_keys.append(key)

        return unique_candidate_keys


    def is_superkey(self, lhs, attributes, candidate_keys):
        # Calculate the closure of lhs
        closure = set(lhs)
        while True:
            new_attributes = closure.copy()
            for fd in self.fds:
                if set(fd['lhs']).issubset(closure):
                    new_attributes.update(fd['rhs'])
            if new_attributes == closure:
                break
            closure = new_attributes

        # If closure covers all attributes, lhs is a superkey
        return closure == set(attributes)



    def is_join_dependency_satisfied(self, lhs, rhs, table):
        
        # Filter out any attributes in rhs that are not present in the table
        filtered_rhs = [attr for attr in rhs if attr in table['attributes']]
        
        if not filtered_rhs:  # If there are no valid RHS attributes, treat as satisfied
            return True
        
        try:
            # Check if the table satisfies the join dependency
            lhs_indices = [table['attributes'].index(attr) for attr in lhs]
            rhs_indices = [table['attributes'].index(attr) for attr in filtered_rhs]

            # Logic to check if join dependency is satisfied (this may involve checking unique combinations)
            lhs_values = {(tuple(row[i] for i in lhs_indices)) for row in table['rows']}
            rhs_values = {(tuple(row[i] for i in rhs_indices)) for row in table['rows']}
            
            return len(lhs_values) == len(rhs_values)

        except ValueError as e:
            print(f"Error checking join dependency {lhs} -> {filtered_rhs} for table {table['table_name']}: {e}")
            return False

    def apply_5NF(self, relation, nf_history):
        
        print(f"\nChecking table {relation['table_name']} for 5NF violations.")

        # Track whether any join dependencies have been violated
        has_violations = False

        # Loop through all previously normalized relations (decomposed tables)
        for table in self.normalized_relations:
            print(f"Checking table {table['table_name']} for join dependencies.")

            for join_dep in self.join_dependencies:
                lhs, rhs = join_dep['lhs'], join_dep['rhs']

                # Check if any attributes in lhs or rhs are not in the current table's attributes
                if not all(attr in table['attributes'] for attr in lhs) or not any(attr in table['attributes'] for attr in rhs):
                    continue  # Skip this join dependency if it can't be checked

                if not self.is_join_dependency_satisfied(lhs, rhs, table):
                    print(f"\nTable {table['table_name']} violates 5NF (join dependency: {join_dep}). Decomposing.")

                    # Decompose the relation based on the join dependency
                    new_tables = self.decompose_5NF(table, lhs, rhs)
                    self.normalized_relations.extend(new_tables)  # Add new tables to normalized relations

                    has_violations = True
                    break  # Break after the first violation and recheck the same table

            if has_violations:
                break  # Exit the loop if we found any violations

        if not has_violations:
            print(f"All join dependencies satisfied for table {relation['table_name']}.")


    def decompose_5NF(self, table, lhs, rhs):
       
        decomposed_tables = []
        
        # Gather unique combinations of lhs values
        lhs_values = set(tuple(row[table['attributes'].index(attr)] for attr in lhs) for row in table['rows'])
        
        for value in lhs_values:
            # Create a new table for each unique value of lhs
            decomposed_table = {
                'table_name': f"{table['table_name']}_5NF_{value}",
                'attributes': list(rhs) + list(lhs),
                'rows': [],
                'primary_key': list(lhs)  # Assuming lhs is the primary key in the new table
            }
            
            # Populate the new table's rows
            for row in table['rows']:
                if tuple(row[table['attributes'].index(attr)] for attr in lhs) == value:
                    new_row = [row[table['attributes'].index(attr)] for attr in rhs] + list(value)
                    decomposed_table['rows'].append(new_row)
            
            decomposed_tables.append(decomposed_table)
            nf_history['5NF'].append(new_relation)

        return decomposed_tables if decomposed_tables else False

    def create_join_table_rows(self, lhs, rhs, table):
       
        lhs_indices = [table['attributes'].index(attr) for attr in lhs]
        rhs_indices = [table['attributes'].index(attr) for attr in rhs]
        
        new_rows = []
        for row in table['rows']:
            new_row = [row[idx] for idx in lhs_indices] + [row[idx] for idx in rhs_indices]
            new_rows.append(new_row)
        
        return new_rows

    def validate_mvd(self, mvd, relation):
       
        lhs, rhs = mvd["lhs"], mvd["rhs"]
        return True  # Return True if MVD is valid, otherwise False

    def print_table(self, relation, step_description):
        
        print(f"\n{step_description} for {relation['table_name']}:")
        df = pd.DataFrame(relation['rows'], columns=relation['attributes'])
        print(df.to_string(index=False))  # Print without row index

def parse_input(file_path: str):
   
    with open(file_path, 'r') as f:
        data = json.load(f)
    relations = data["relations"]
    fds = data["functional_dependencies"]
    mvds = data["mvds"]
    target_nf = data["target_normal_form"]
    return relations, fds, mvds, target_nf


if __name__ == "__main__":
    # Ask user for the input file path
    file_path = input("Enter the JSON file name (in the current directory): ")
    
    try:
        relations, fds, mvds, target_nf = parse_input(file_path)
        normalizer = Normalizer(relations, fds, mvds, target_nf)
        normalized_relations = normalizer.normalize()

      
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")

Enter the JSON file name (in the current directory):  output.json



Normalizing table: CoffeeShopData
Original table before normalization:

Original for CoffeeShopData:
 OrderID    Date        PromocodeUsed TotalCost TotalDrinkCost TotalFoodCost  CustomerID CustomerName  DrinkID                DrinkName DrinkSize  DrinkQuantity Milk                        DrinkIngredient DrinkAllergen  FoodID          FoodName  FoodQuantity                    FoodIngredient FoodAllergen
    1001 6/30/24                 NONE    $7.25          $7.25         $0.00            1  Alice Brown        1              Caffe Latte    Grande              1   ND                   {Espresso, Oat Milk}        {Oat}        0               NaN             0                              NONE         NONE
    1002 6/30/26            SUMMERFUN    $9.98          $5.99         $3.99            2 David Miller        2   Iced Caramel Macchiato      Tall              2    D   {Expresso, Vanilla Syrup, Milk, Ice} {Dairy, Nuts}       3 Blueberry Muffin              1 {Flour, Sugar, Blueberries,